# Notebook 5.2 Speech Recognition

Speech recognition, also known as automatic speech recognition (ASR), is a technology that converts spoken words into written format or executes specific actions based on verbal commands. It involves machine learning models that analyze speech patterns, phonetics, and language structures to accurately transcribe and understand human speech.

[Whisper](https://openai.com/research/whisper), published by OpenAI, is a popular open-source model for both ASR and speech translation. This means that Whisper has the capability to transcribe speech in multiple languages and facilitate translation from those languages into English.

Due to its underlying Transformer-based encoder-decoder architecture, Whisper can be optimized effectively with BigDL-LLM INT4 optimizations. In this tutorial, we will guide you through building a speech recognition application on BigDL-LLM optimized Whisper model that can transcribe/translate audio files into text.

## 5.2.1 Install Packages

Follow instructions in [Chapter 2](../ch_2_Environment_Setup/README.md) to setup your environment if you haven't done so. Then install bigdl-llm:

In [ ]:
!pip install bigdl-llm[all]

Due to the requirement to process audio file, you will also need to install the `librosa` package for audio analysis.

In [ ]:
!pip install -U librosa

## 5.2.2 Download Audio Files

To begin, let's prepare some audio files. As an example, you can download audio files from the multilingual [common_voice](https://huggingface.co/datasets/common_voice/viewer/en/train) datasets. Here, one English audio file and one Chinese audio file have been randomly selected. Feel free to choose different audio files according to your preferences.

In [1]:
!wget -O audio_en.mp3 https://datasets-server.huggingface.co/assets/common_voice/--/en/train/5/audio/audio.mp3
!wget -O audio_zh.mp3 https://datasets-server.huggingface.co/assets/common_voice/--/zh-CN/train/2/audio/audio.mp3

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


You could play the successfully-downloaded audio:

In [4]:
import IPython

IPython.display.display(IPython.display.Audio("en.mp3"))
IPython.display.display(IPython.display.Audio("ch.mp3"))

## 5.2.3 Load Pretrained Whisper Model

Now, let's load a pretrained Whisper model, e.g. [whisper-medium](https://huggingface.co/openai/whisper-medium) as an example. OpenAI has released pretrained Whisper models in various sizes (including [whisper-small](https://huggingface.co/openai/whisper-small), [whisper-tiny](https://huggingface.co/openai/whisper-tiny), etc.), allowing you to choose the one that best fits your requirements. 

Simply use one-line `transformers`-style API in `bigdl-llm` to load `whisper-medium` with INT4 optimizations (by specifying `load_in_4bit=True`) as follows. Please note that model class `AutoModelForSpeechSeq2Seq` is used for Whisper:

In [2]:
from bigdl.llm.transformers import AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained(pretrained_model_name_or_path="./model/",
                                                  load_in_4bit=True)

2023-11-16 11:28:56,693 - INFO - Converting the current model to sym_int4 format......


## 5.2.4 Load Whisper Processor

A Whisper processor is also needed for both audio pre-processing, and post-processing model outputs from tokens to texts. Just use the official `transformers` API to load `WhisperProcessor`:

In [3]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(pretrained_model_name_or_path="./model/")

## 5.2.5 Transcribe English Audio

Once you have optimized the Whisper model using BigDL-LLM with INT4 optimization and loaded the Whisper processor, you are ready to begin transcribing the audio through model inference.

Let's start with the English audio file `audio_en.mp3`. Before we feed it into Whisper processor, we need to extract sequence data from raw speech waveform:

In [9]:
import librosa

data_en, sample_rate_en = librosa.load("en.mp3", sr=16000)
print(sample_rate_en)
print(int(data_en.shape[0])/16000)

16000
8.7771875


> **Note**
>
> For `whisper-medium`, its `WhisperFeatureExtractor` (part of `WhisperProcessor`) extracts features from audio using a 16,000Hz sampling rate by default. It's important to load the audio file at the sample sampling rate with model's `WhisperFeatureExtractor` for precise recognition.

We can then proceed to transcribe the audio file based on the sequence data, using exactly the same way as using official `transformers` API:

In [10]:
import torch
import time

# define task type
forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

with torch.inference_mode():
    # extract input features for the Whisper model  mel-filter bank features
    input_features = processor(data_en, sampling_rate=sample_rate_en, return_tensors="pt").input_features

    # predict token ids for transcription
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()

    # decode token ids into texts
    transcribe_str = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'English Transcription', '-'*20)
    print(transcribe_str)

[(1, 50259), (2, 50359), (3, 50363)]


C:\Users\deep\anaconda3\envs\bigdl\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Inference time: 7.836472034454346 s
-------------------- English Transcription --------------------
[' And many diseases are also caused by the various additives in food. We often heard that a bag of cheese may contain 120 kinds of additives.']


> **Note**
>
> `forced_decoder_ids` defines the context token for different language and task (transcribe or translate). If it is set to `None`, Whisper will automatically predict them.


## 5.2.6 Transcribe Chinese Audio and Translate to English

Then let's move to the Chinese audio `audio_zh.mp3`. The training corpus of Whisper includes 680,000 hours of audio and covers over 90 languages. This allows us to achieve translation from it to English. Whisper can transcribe multilingual audio, and translate them into English. The only difference here is to define specific context token through `forced_decoder_ids`:

In [12]:
# extract sequence data
data_zh, sample_rate_zh = librosa.load("zh.mp3", sr=16000)

# define Chinese transcribe and translation task
forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task="transcribe")

with torch.inference_mode():
    input_features = processor(data_zh, sampling_rate=sample_rate_zh, return_tensors="pt").input_features
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()
    translate_str = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'Chinese to English Translation', '-'*20)
    print(translate_str)

Inference time: 8.468757152557373 s
-------------------- Chinese to English Translation --------------------
['是对经济社会发展情况的一次全面体验对于摸清家底反映发展成效具有重大而深远的意义第5次全国经济普查标准是']


## 5.2.7 What's Next?

In the upcoming chapter, we will explore the usage of BigDL-LLM in conjunction with langchain, a framework designed for developing applications with language models. With langchain integration, application development process could be simplified.